In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# KNN (K-Nearest Neighbors – Classification/Regression)

## 1. Dự đoán bệnh tiểu đường

Input: tuổi, cân nặng, mức đường huyết, huyết áp.

KNN sẽ tìm các bệnh nhân gần nhất trong dữ liệu huấn luyện để đoán xem người mới có khả năng bị tiểu đường hay không.

In [2]:
# 1) Đọc dữ liệu
df = pd.read_csv("diabetes_knn.csv")
X = df[["Age", "Weight", "Glucose", "BloodPressure"]]
y = df["Diabetes"]

# 2) Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# 3) Tạo pipeline: Chuẩn hóa (StandardScaler) + KNN(k=5)
model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5, weights="distance"))
])

# 4) Huấn luyện
model.fit(X_train, y_train)

# 5) Đánh giá
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8333333333333334
Confusion matrix:
 [[12  3]
 [ 2 13]]
              precision    recall  f1-score   support

           0       0.86      0.80      0.83        15
           1       0.81      0.87      0.84        15

    accuracy                           0.83        30
   macro avg       0.83      0.83      0.83        30
weighted avg       0.83      0.83      0.83        30



In [3]:
# Ví dụ bệnh nhân mới:
new_patient = pd.DataFrame([{
    "Age": 45,
    "Weight": 72,       # kg
    "Glucose": 145,     # mg/dL
    "BloodPressure": 88 # mmHg (tâm trương)
}])

proba = model.predict_proba(new_patient)[0][1]  # xác suất bị tiểu đường (class=1)
pred  = model.predict(new_patient)[0]

print("Predicted label (0=No, 1=Yes):", int(pred))
print("Probability of Diabetes:", round(proba, 3))

Predicted label (0=No, 1=Yes): 1
Probability of Diabetes: 0.889


In [4]:
param_grid = {
    "knn__n_neighbors": [3,5,7,9,11],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1,2]  # Manhattan vs Euclidean
}

grid = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)
grid.fit(X, y)
print("Best params:", grid.best_params_)
print("Best CV score:", grid.best_score_)

Best params: {'knn__n_neighbors': 7, 'knn__p': 1, 'knn__weights': 'distance'}
Best CV score: 0.7916666666666666


## 2. Nhận diện chữ viết tay (MNIST dataset)

Input: ảnh chữ số viết tay (0–9).

KNN tìm những ảnh gần nhất trong tập huấn luyện.

Nếu phần lớn hàng xóm là số 8 → mẫu mới được dự đoán là số 8.

### Cách 1 — Dùng Keras (tiện, nhanh)

In [5]:
from tensorflow.keras.datasets import mnist
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from PIL import Image

In [6]:
# 1) Tải MNIST (60k train, 10k test)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 2) Chuyển ảnh 28x28 -> vector 784 & chuẩn hóa [0,1]
X_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
X_test  = x_test.reshape(-1, 28*28).astype("float32") / 255.0

# (Tùy chọn) Dùng subset cho nhanh: ~10k mẫu
X_sub, _, y_sub, _ = train_test_split(X_train, y_train, train_size=10000, stratify=y_train, random_state=42)

# 3) KNN
knn = KNeighborsClassifier(n_neighbors=3, weights="distance", n_jobs=-1)
knn.fit(X_sub, y_sub)

# 4) Đánh giá
y_pred = knn.predict(X_test[:2000])  # chấm 2k mẫu cho nhanh
print("Accuracy:", accuracy_score(y_test[:2000], y_pred))
print(classification_report(y_test[:2000], y_pred))

c:\Users\CaRot\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\CaRot\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\CaRot\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\CaRot\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1039

Accuracy: 0.9225
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       175
           1       0.90      1.00      0.95       234
           2       0.99      0.88      0.93       219
           3       0.90      0.92      0.91       207
           4       0.95      0.91      0.93       217
           5       0.91      0.91      0.91       179
           6       0.95      0.97      0.96       178
           7       0.90      0.90      0.90       205
           8       0.95      0.82      0.88       192
           9       0.84      0.93      0.88       194

    accuracy                           0.92      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.93      0.92      0.92      2000



In [7]:
# Lưu 5k ảnh train: 784 cột pixel + 1 cột label
N = 5000
df = pd.DataFrame(X_train[:N])
df["label"] = y_train[:N]
df.to_csv("mnist_5k_flat.csv", index=False)
print("Đã lưu mnist_5k_flat.csv")

Đã lưu mnist_5k_flat.csv


In [8]:
import matplotlib.pyplot as plt

# Lấy 1 ảnh số viết tay từ tập test
sample_img = x_test[0].reshape(28,28) * 255
plt.imsave("digit.png", sample_img, cmap="gray")

In [9]:
def load_digit_image(path):
    # Đọc ảnh, chuyển grayscale, resize 28x28, scale [0,1]
    img = Image.open(path).convert("L").resize((28,28))
    arr = np.array(img).astype("float32") / 255.0
    return arr.reshape(1, 28*28)

x_new = load_digit_image("digit.png")   # ảnh số viết tay nền trắng/đen
pred = knn.predict(x_new)[0]
proba = knn.predict_proba(x_new).max()
print("Dự đoán:", pred, "| Độ tự tin:", round(proba, 3))

Dự đoán: 7 | Độ tự tin: 0.668


### Cách 2 — Dùng OpenML (chuẩn MNIST_784, không cần TensorFlow)

In [10]:
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [11]:
# 1) Tải MNIST_784
mnist = fetch_openml("mnist_784", version=1, as_frame=False)  # X: (70000, 784), y: str
X = mnist.data.astype("float32") / 255.0
y = mnist.target.astype(int)

# 2) Chia tập (dùng 12k để demo KNN nhanh)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=12000, test_size=2000, stratify=y, random_state=42
)

# 3) KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3, weights="distance", n_jobs=-1)
knn.fit(X_train, y_train)

# 4) Đánh giá
pred = knn.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))

Accuracy: 0.9515


In [12]:
df = pd.DataFrame(X_train)
df["label"] = y_train
df.to_csv("mnist_subset_train.csv", index=False)
print("Đã lưu mnist_subset_train.csv")

Đã lưu mnist_subset_train.csv


### Cách 3 (NHẸ, chạy siêu nhanh) — sklearn.datasets.load_digits (8×8, không phải MNIST 28×28)

In [13]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [14]:
digits = load_digits()  # 1797 ảnh, 8x8 = 64 pixels
X = digits.data / 16.0  # scale [0,1]
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

knn = KNeighborsClassifier(n_neighbors=3, weights="distance")
knn.fit(X_train, y_train)
print("Accuracy:", accuracy_score(y_test, knn.predict(X_test)))

Accuracy: 0.9844444444444445
